In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

%matplotlib inline

In [ ]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''select * from factihealth.mimic.mortality_data''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)
    

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

In [ ]:
def pre_processing_mortality_feature_list(data):
        # dropping columns which are not required
    data=master_data  
    data.drop(['ph_dipstick','ethnicity','admission_type','mixed_venous_o2_sat','first_careunit','age_bucket','icu_los','edadmittime'],axis = 1, inplace = True)

    # Cleaning the data by adding limits to remove unrealistic data due to human/techincal errors

    condition_1 = data['non_invasive_blood_pressure_diastolic'] > 200
    data.loc[condition_1, 'non_invasive_blood_pressure_diastolic'] = data.loc[condition_1, 'non_invasive_blood_pressure_mean']
    data = data[(data['heart_rate_mean'] >= 0) & (data['heart_rate_mean'] <= 500)]
    data = data[(data['respiratory_rate'] >= 0) & (data['respiratory_rate'] <= 100)]
    data = data[(data['temperature_fahrenheit'] >= 0) & (data['temperature_fahrenheit'] <= 250)]
    data = data[(data['admission_weight_lbs'] >= 0) & (data['admission_weight_lbs'] <= 1000)]
    data = data[(data['o2_saturation_pulseoxymetry2'] >= 0) & (data['o2_saturation_pulseoxymetry2'] <= 100)]
    data = data[(data['height_cm'] >= 0) & (data['height_cm'] <= 300)]
    data.drop(['non_invasive_blood_pressure_mean'],axis = 1, inplace = True)

    df_encoded = pd.get_dummies(data, columns=['admission_type_combined', 'first_careunit_combined','ethnicity_combined','gender'])

    mean_value = df_encoded['non_invasive_blood_pressure_diastolic'].mean()

    # Replace NaNs in the column with the calculated mean value
    df_encoded['non_invasive_blood_pressure_diastolic'].fillna(mean_value, inplace=True)
    df_encoded['gcs_eye_opening'].fillna(0, inplace=True)
    df_encoded['gcs_verbal_response'].fillna(0, inplace=True)
    df_encoded['gcs_motor_response'].fillna(0, inplace=True)

    df_encoded.drop(columns=['subject_id','hadm_id'],inplace = True)
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

In [ ]:
data = pre_processing_mortality(master_data)

In [ ]:
data.info()

In [ ]:
X = data.drop('hospital_expire_flag', axis=1)
y = data['hospital_expire_flag']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5,8],
    # Add other parameters here
}

# Create a base model
xgb_classifier = xgb.XGBClassifier(random_state=42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

In [ ]:
import pickle
# Save the model to disk
filename = 'icu_patients_mortality_prediction.pkl'  # Change this path if you need to save it elsewhere
pickle.dump(grid_search.best_estimator_, open(filename, 'wb'))

print(f"Model saved to {filename}")

# For Streamlit

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

%matplotlib inline

In [2]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    cur.execute('''select * from factihealth.mimic.mortality_data''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    master_data_for_all_features = pd.DataFrame(rows, columns=column_names) 
    
    # Execute a query
    cur.execute(
                '''select 
                        distinct b.*,
                                to_date(REPLACE(b.edadmittime,'2130','2023'),'YYYY-MM-DD') edadmit_time
                    from 
                        factihealth.mimic.ecg_model_dataset a
                    inner join 
                        factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
                    where date_part(year,edadmittime) = 2130
                        and date_part(year,ecgtime) = 2130
                        and ecgtime between edadmittime and eddischargetime
                    order by ecgtime,a.subject_id
                    ''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (143, 29)


In [3]:
def pre_processing_mortality_feature_list(data):
        # dropping columns which are not required
    data.drop(['ph_dipstick','ethnicity','admission_type','mixed_venous_o2_sat','first_careunit','age_bucket','icu_los','edadmittime','eddischargetime'],axis = 1, inplace = True)

    # Cleaning the data by adding limits to remove unrealistic data due to human/techincal errors

    condition_1 = data['non_invasive_blood_pressure_diastolic'] > 200
    data.loc[condition_1, 'non_invasive_blood_pressure_diastolic'] = data.loc[condition_1, 'non_invasive_blood_pressure_mean']
    data = data[(data['heart_rate_mean'] >= 0) & (data['heart_rate_mean'] <= 500)]
    data = data[(data['respiratory_rate'] >= 0) & (data['respiratory_rate'] <= 100)]
    data = data[(data['temperature_fahrenheit'] >= 0) & (data['temperature_fahrenheit'] <= 250)]
    data = data[(data['admission_weight_lbs'] >= 0) & (data['admission_weight_lbs'] <= 1000)]
    data = data[(data['o2_saturation_pulseoxymetry2'] >= 0) & (data['o2_saturation_pulseoxymetry2'] <= 100)]
    data = data[(data['height_cm'] >= 0) & (data['height_cm'] <= 300)]
    data.drop(['non_invasive_blood_pressure_mean'],axis = 1, inplace = True)

    df_encoded = pd.get_dummies(data, columns=['admission_type_combined', 'first_careunit_combined','ethnicity_combined','gender'])

    mean_value = df_encoded['non_invasive_blood_pressure_diastolic'].mean()

    # Replace NaNs in the column with the calculated mean value
    df_encoded['non_invasive_blood_pressure_diastolic'].fillna(mean_value, inplace=True)
    df_encoded['gcs_eye_opening'].fillna(0, inplace=True)
    df_encoded['gcs_verbal_response'].fillna(0, inplace=True)
    df_encoded['gcs_motor_response'].fillna(0, inplace=True)

    df_encoded.drop(columns=['subject_id','hadm_id','hospital_expire_flag'],inplace = True)
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

In [4]:
def pre_processing_mortality(data):    
    # dropping columns which are not required
    data.drop(['ph_dipstick','hospital_expire_flag','admission_type','mixed_venous_o2_sat','first_careunit','age_bucket','icu_los','edadmittime','eddischargetime'],axis = 1, inplace = True)

    # Cleaning the data by adding limits to remove unrealistic data due to human/techincal errors

    condition_1 = data['non_invasive_blood_pressure_diastolic'] > 200
    data.loc[condition_1, 'non_invasive_blood_pressure_diastolic'] = data.loc[condition_1, 'non_invasive_blood_pressure_mean']
    data = data[(data['heart_rate_mean'] >= 0) & (data['heart_rate_mean'] <= 500)]
    data = data[(data['respiratory_rate'] >= 0) & (data['respiratory_rate'] <= 100)]
    data = data[(data['temperature_fahrenheit'] >= 0) & (data['temperature_fahrenheit'] <= 250)]
    data = data[(data['admission_weight_lbs'] >= 0) & (data['admission_weight_lbs'] <= 1000)]
    data = data[(data['o2_saturation_pulseoxymetry2'] >= 0) & (data['o2_saturation_pulseoxymetry2'] <= 100)]
    data = data[(data['height_cm'] >= 0) & (data['height_cm'] <= 300)]
    data.drop(['non_invasive_blood_pressure_mean'],axis = 1, inplace = True)
    
    df_encoded = pd.get_dummies(data, columns=['admission_type_combined', 'first_careunit_combined','ethnicity_combined','gender'])
    
    mean_value = df_encoded['non_invasive_blood_pressure_diastolic'].mean()

    # Replace NaNs in the column with the calculated mean value
    df_encoded['non_invasive_blood_pressure_diastolic'].fillna(mean_value, inplace=True)
    df_encoded['gcs_eye_opening'].fillna(0, inplace=True)
    df_encoded['gcs_verbal_response'].fillna(0, inplace=True)
    df_encoded['gcs_motor_response'].fillna(0, inplace=True)
    
    #df_encoded.drop(columns=['subject_id','hadm_id'],inplace = True)
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

In [5]:
features_data = pre_processing_mortality_feature_list(master_data_for_all_features)
mortal_data = pre_processing_mortality(master_data)
df_merge = mortal_data.copy()
df_merge['index'] = range(0, len(df_merge))
df_merge = df_merge[['index','subject_id','age','height_cm','ethnicity','edadmit_time']]
df_merge

,index,subject_id,age,height_cm,ethnicity,edadmit_time
2,0,13494259,77,170.0,WHITE - OTHER EUROPEAN,2023-01-09
5,1,18274981,78,166.0,UNKNOWN,2023-01-14
6,2,15311489,57,178.0,BLACK/CAPE VERDEAN,2023-01-18
7,3,11115658,59,173.0,WHITE,2023-01-22
8,4,18483564,32,173.5,WHITE,2023-01-21
...,...,...,...,...,...,...
131,71,13103745,59,173.0,BLACK/AFRICAN AMERICAN,2023-12-09
132,72,12241660,21,178.0,HISPANIC/LATINO - COLUMBIAN,2023-12-10
134,73,15102490,44,178.0,WHITE,2023-12-17
139,74,14992632,76,178.0,WHITE,2023-12-26


In [6]:
def post_process_mortality(data_1,data_2):
    
    data = data_1
    features_data = data_2
    
    missing_columns = set(features_data.columns)-set(data.columns)

    if missing_columns:
        # Add missing columns and fill them with zeros
        for column in missing_columns:
            data[column] = 0  # Fill with zeros
            
    data.drop(columns=['subject_id','hadm_id','ethnicity','edadmit_time'],inplace = True)
    data=data.sort_index(axis=1)    
    
    return data

In [7]:
final_data = post_process_mortality(mortal_data,features_data)

In [8]:
final_data.head(5)

,admission_type_combined_Elective,admission_type_combined_Emergency,admission_type_combined_Observation,admission_type_combined_Surgical Same Day Admission,admission_weight_lbs,age,ethnicity_combined_AMERICAN INDIAN/ALASKA NATIVE,ethnicity_combined_ASIAN,ethnicity_combined_BLACK/AFRICAN,ethnicity_combined_HISPANIC OR LATINO,...,gcs_motor_response,gcs_verbal_response,gender_F,gender_M,heart_rate_mean,height_cm,non_invasive_blood_pressure_diastolic,o2_saturation_pulseoxymetry2,respiratory_rate,temperature_fahrenheit
2,0,0,1,0,189.199997,77,0,0,0,0,...,5.444444,4.555556,0,1,76.607143,170.0,49.315789,93.678571,14.725490,98.100000
5,0,1,0,0,116.599998,78,0,0,0,0,...,6.000000,3.323077,1,0,89.338129,166.0,57.420664,94.383513,18.760870,98.025000
6,0,0,0,1,158.399994,57,0,0,1,0,...,5.166667,4.333333,0,1,78.300000,178.0,68.777778,99.344828,14.400000,98.599998
7,0,1,0,0,167.199997,59,0,0,0,0,...,6.000000,5.000000,1,0,57.962963,173.0,62.629630,98.961538,14.653846,97.528573
8,0,1,0,0,112.142858,32,0,0,0,0,...,5.952381,3.380952,0,1,107.202312,173.5,75.500000,98.680233,24.202312,97.994595


In [9]:
import pickle
with open('icu_patients_mortality_prediction.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Make predictions using the loaded model and obtain prediction probabilities
prediction_probabilities = loaded_model.predict_proba(final_data)

# Create a DataFrame from the prediction probabilities with a reset index
probabilities_df = pd.DataFrame(prediction_probabilities).reset_index()
mortality_pred = pd.merge(df_merge,probabilities_df,on='index',how ='inner')
mortality_pred.drop(columns = 'index',inplace = True)
mortality_pred.rename(columns={0:'Alive',1:'Dead'},inplace = True)
mortality_pred

,subject_id,age,height_cm,ethnicity,edadmit_time,Alive,Dead
0,13494259,77,170.0,WHITE - OTHER EUROPEAN,2023-01-09,0.974777,0.025223
1,18274981,78,166.0,UNKNOWN,2023-01-14,0.909095,0.090905
2,15311489,57,178.0,BLACK/CAPE VERDEAN,2023-01-18,0.999995,0.000005
3,11115658,59,173.0,WHITE,2023-01-22,0.999978,0.000022
4,18483564,32,173.5,WHITE,2023-01-21,0.992985,0.007015
...,...,...,...,...,...,...,...
71,13103745,59,173.0,BLACK/AFRICAN AMERICAN,2023-12-09,0.998696,0.001304
72,12241660,21,178.0,HISPANIC/LATINO - COLUMBIAN,2023-12-10,0.999559,0.000441
73,15102490,44,178.0,WHITE,2023-12-17,0.999597,0.000403
74,14992632,76,178.0,WHITE,2023-12-26,0.999419,0.000581


In [29]:
mortality_pred = pd.merge(df_merge,probabilities_df,on='index',how ='inner')
mortality_pred.drop(columns = 'index',inplace = True)
mortality_pred.rename(columns={0:'Alive',1:'Dead'},inplace = True)
mortality_pred

,subject_id,age,height_cm,ethnicity,edadmit_time,Alive,Dead
0,13494259,77,170.0,WHITE - OTHER EUROPEAN,2023-01-09,0.974777,0.025223
1,18274981,78,166.0,UNKNOWN,2023-01-14,0.909095,0.090905
2,15311489,57,178.0,BLACK/CAPE VERDEAN,2023-01-18,0.999995,0.000005
3,11115658,59,173.0,WHITE,2023-01-22,0.999978,0.000022
4,18483564,32,173.5,WHITE,2023-01-21,0.992985,0.007015
...,...,...,...,...,...,...,...
71,13103745,59,173.0,BLACK/AFRICAN AMERICAN,2023-12-09,0.998696,0.001304
72,12241660,21,178.0,HISPANIC/LATINO - COLUMBIAN,2023-12-10,0.999559,0.000441
73,15102490,44,178.0,WHITE,2023-12-17,0.999597,0.000403
74,14992632,76,178.0,WHITE,2023-12-26,0.999419,0.000581


In [30]:
# Exclude the index column from the DataFrame
mortality_pred1 = mortality_pred.reset_index().copy()
mortality_pred1.index = [''] * len(mortality_pred1)

mortality_pred1['index'] = mortality_pred1['index']+1

mortality_pred['Alive'] = (mortality_pred['Alive']*100).astype(float)
mortality_pred1['Dead']  = (mortality_pred1['Dead'] *100).astype(float)

mortality_pred1['Alive'] = mortality_pred1['Alive'].apply(lambda x: f'{x:.1%}')
mortality_pred1['Dead'] = mortality_pred1['Dead'].apply(lambda x: f'{x:.1%}')


# Display the modified DataFrame without the index using st.dataframe()
mortality_pred1

,index,subject_id,age,height_cm,ethnicity,edadmit_time,Alive,Dead
,1,13494259,77,170.0,WHITE - OTHER EUROPEAN,2023-01-09,97.5%,2.52
,2,18274981,78,166.0,UNKNOWN,2023-01-14,90.9%,9.09
,3,15311489,57,178.0,BLACK/CAPE VERDEAN,2023-01-18,100.0%,0.00
,4,11115658,59,173.0,WHITE,2023-01-22,100.0%,0.00
,5,18483564,32,173.5,WHITE,2023-01-21,99.3%,0.70
...,...,...,...,...,...,...,...,...
,72,13103745,59,173.0,BLACK/AFRICAN AMERICAN,2023-12-09,99.9%,0.13
,73,12241660,21,178.0,HISPANIC/LATINO - COLUMBIAN,2023-12-10,100.0%,0.04
,74,15102490,44,178.0,WHITE,2023-12-17,100.0%,0.04
,75,14992632,76,178.0,WHITE,2023-12-26,99.9%,0.06


In [39]:
mortality_pred1.rename(columns={'height_cm':'Height','subject_id':'Patient ID',
                               'ethnicity':'Ethnicity','edadmit_time':'ED: Admission Date',
                               'Alive':'Probabilty of Being Alive',
                               'Dead':'Probability of being Dead'},inplace=True)
mortality_pred1

,index,Patient ID,age,Height,Ethnicity,ED: Admission Date,Probabilty of Being Alive,Probability of being Dead
,1,13494259,77,170.0,WHITE - OTHER EUROPEAN,2023-01-09,97.5%,2.52
,2,18274981,78,166.0,UNKNOWN,2023-01-14,90.9%,9.09
,3,15311489,57,178.0,BLACK/CAPE VERDEAN,2023-01-18,100.0%,0.00
,4,11115658,59,173.0,WHITE,2023-01-22,100.0%,0.00
,5,18483564,32,173.5,WHITE,2023-01-21,99.3%,0.70
...,...,...,...,...,...,...,...,...
,72,13103745,59,173.0,BLACK/AFRICAN AMERICAN,2023-12-09,99.9%,0.13
,73,12241660,21,178.0,HISPANIC/LATINO - COLUMBIAN,2023-12-10,100.0%,0.04
,74,15102490,44,178.0,WHITE,2023-12-17,100.0%,0.04
,75,14992632,76,178.0,WHITE,2023-12-26,99.9%,0.06


In [43]:
mortality_pred1[['Patient ID','Probabilty of Being Alive','Probability of being Dead']]

,Patient ID,Probabilty of Being Alive,Probability of being Dead
,13494259,97.5%,2.52
,18274981,90.9%,9.09
,15311489,100.0%,0.00
,11115658,100.0%,0.00
,18483564,99.3%,0.70
...,...,...,...
,13103745,99.9%,0.13
,12241660,100.0%,0.04
,15102490,100.0%,0.04
,14992632,99.9%,0.06
